# Intro

This notebook is to calcuate ATLN and ENSO indices and get a list of years for the phase of each on the CESM LENS2 project

# Imports

In [4]:
import numpy as np
import pandas as pd
import xarray as xr
# import netCDF4
# import matplotlib
# import matplotlib.pyplot as plt
# import datetime
import cftime
import dask
# from ncar_jobqueue import NCARCluster
from dask_jobqueue import PBSCluster
from dask.distributed import Client
import xeofs as xe
import glob
# from tqdm import tqdm, trange
# import time
# import scipy

# Start PBSCluster

In [11]:
# Create a PBS cluster object
cluster = PBSCluster(account='P93300313',
                     job_name='ATLN-ENSO-CESMLE2',
                     cores=1,
                     memory='8GiB',
                     processes=1,
                     walltime='02:00:00',
                     queue='casper',
                     interface='ext',
                     n_workers=1)

# dont scale unless using LE
# cluster.scale(10)

client = Client(cluster)
client

/glade/u/home/acruz/.conda/envs/EOFa_2025/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40563 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/acruz/proxy/40563/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/acruz/proxy/40563/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.210:44589,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/acruz/proxy/40563/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [19]:
cluster.scale(2)

In [30]:
client.shutdown()
cluster.workers

{}

# Data Imports

In [21]:
# files = '/glade/campaign/collections/rda/data/d010092/ocn/monthly/cesm2LE-historical-cmip6-TEMP.zarr'
# files = glob.glob('/glade/campaign/cgd/cesm/CESM2-LE/timeseries/atm/proc/tseries/month_1/SST/*.nc')
# start with hadisst files
ds = xr.open_dataset('/glade/campaign/collections/rda/data/d277003/HadISST_sst.nc.gz')

In [22]:
ds

<xarray.Dataset> Size: 482MB
Dimensions:    (time: 1861, nv: 2, latitude: 180, longitude: 360)
Coordinates:
  * time       (time) datetime64[ns] 15kB 1870-01-16T11:59:59.505615234 ... 2...
  * latitude   (latitude) float32 720B 89.5 88.5 87.5 86.5 ... -87.5 -88.5 -89.5
  * longitude  (longitude) float32 1kB -179.5 -178.5 -177.5 ... 178.5 179.5
Dimensions without coordinates: nv
Data variables:
    time_bnds  (time, nv) float32 15kB ...
    sst        (time, latitude, longitude) float32 482MB ...
Attributes:
    Title:                      Monthly version of HadISST sea surface temper...
    description:                HadISST 1.1 monthly average sea surface tempe...
    institution:                Met Office Hadley Centre
    source:                     HadISST
    reference:                  Rayner, N. A., Parker, D. E., Horton, E. B., ...
    Conventions:                CF-1.0
    history:                    3/3/2025 converted to netcdf from pp format
    supplementary_information:  Updates and supplementary information will be...
    comment:                    Data restrictions: for academic research use ...

# select data

In [27]:
ATL_hadisst = ds.sel(latitude=slice(10, -10), longitude=slice(-40, 20))
ENSO34_hadisst = ds.sel(latitude=slice(5, -5), longitude=slice(-170, -120))

# ATL3 and Niño3.4 indices

In [29]:
# get weight of latitudes
area_weight = np.cos(np.deg2rad(ds.latitude))
area_weight.name = "weights"

# weight SSTs
weighted_sst = ENSO34_hadisst.weighted(area_weight)

# calculate mean SST in the area
sst_mean = weighted_sst.mean(("longitude", "latitude"))
sst_mean

ValueError: Dimension(s) 'longitude' do not exist. Expected one or more of {'nv', 'time', 'latitude'}

## Niño3.4 Index

In [28]:
ATL_hadisst

<xarray.Dataset> Size: 923kB
Dimensions:    (time: 1861, nv: 2, latitude: 6, longitude: 20)
Coordinates:
  * time       (time) datetime64[ns] 15kB 1870-01-16T11:59:59.505615234 ... 2...
  * latitude   (latitude) float32 24B 2.5 1.5 0.5 -0.5 -1.5 -2.5
  * longitude  (longitude) float32 80B -19.5 -18.5 -17.5 ... -2.5 -1.5 -0.5
Dimensions without coordinates: nv
Data variables:
    time_bnds  (time, nv) float32 15kB ...
    sst        (time, latitude, longitude) float32 893kB ...
Attributes:
    Title:                      Monthly version of HadISST sea surface temper...
    description:                HadISST 1.1 monthly average sea surface tempe...
    institution:                Met Office Hadley Centre
    source:                     HadISST
    reference:                  Rayner, N. A., Parker, D. E., Horton, E. B., ...
    Conventions:                CF-1.0
    history:                    3/3/2025 converted to netcdf from pp format
    supplementary_information:  Updates and supplementary information will be...
    comment:                    Data restrictions: for academic research use ...